In [ ]:
! pip install tensorflow tensorflow-gpu opencv-python mediapipe sklearn matplotlib keras

In [1]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import mediapipe as mp
import tensorflow as tf

In [2]:
mp_holistic = mp.solutions.holistic
mp_drawing_utils = mp.solutions.drawing_utils

In [3]:
def mediapipe_detection(frame, model):
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame.flags.writeable = False
    result = model.process(frame)
    frame.flags.writeable = True
    frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
    return frame, result

In [4]:
def extract_landmark(result):
    pose = np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in result.pose_landmarks.landmark]).flatten()\
        if result.pose_landmarks else np.zeros(33 * 4)
    face = np.array([[landmark.x, landmark.y, landmark.z] for landmark in result.face_landmarks.landmark]).flatten() \
        if result.face_landmarks else np.zeros(468 * 3)
    left_hand = np.array([[landmark.x, landmark.y, landmark.z] for landmark in result.left_hand_landmarks.landmark]).flatten() \
        if result.left_hand_landmarks else np.zeros(21 * 3)
    right_hand = np.array([[landmark.x, landmark.y, landmark.z] for landmark in result.right_hand_landmarks.landmark]).flatten() \
        if result.right_hand_landmarks else np.zeros(21 * 3)
    return np.concatenate([pose, face, left_hand, right_hand])

In [5]:
def draw_landmarks(frame, result):
    mp_drawing_utils.draw_landmarks(frame, result.face_landmarks, mp_holistic.FACEMESH_CONTOURS,
                                    mp_drawing_utils.DrawingSpec(color=(80, 110, 10), thickness=1, circle_radius=1),
                                    mp_drawing_utils.DrawingSpec(color=(80, 256, 121), thickness=1, circle_radius=1)
                                    )
    mp_drawing_utils.draw_landmarks(frame, result.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                                    mp_drawing_utils.DrawingSpec(color=(80, 110, 10), thickness=1, circle_radius=1),
                                    mp_drawing_utils.DrawingSpec(color=(80, 256, 121), thickness=1, circle_radius=1)
                                    )
    mp_drawing_utils.draw_landmarks(frame, result.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                                    mp_drawing_utils.DrawingSpec(color=(80, 110, 10), thickness=1, circle_radius=1),
                                    mp_drawing_utils.DrawingSpec(color=(80, 256, 121), thickness=1, circle_radius=1)
                                    )
    mp_drawing_utils.draw_landmarks(frame, result.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                                    mp_drawing_utils.DrawingSpec(color=(80, 110, 10), thickness=1, circle_radius=1),
                                    mp_drawing_utils.DrawingSpec(color=(80, 256, 121), thickness=1, circle_radius=1)
                                    )

In [6]:
capture = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while capture.isOpened():
        return_value, frame = capture.read()
        frame, result = mediapipe_detection(frame, holistic)
        draw_landmarks(frame, result)
        cv2.imshow("Mime by Ridhi and Kartikay", frame)
        if cv2.waitKey(1) & 0xFF == ord('x'):
            break
    capture.release()
    cv2.destroyAllWindows()

In [7]:
data = os.path.join('data')
signs = np.array(['hello.left', 'hello.right', 'thank you.left', 'thank you.right', 'i love you.left', 'i love you.right', 'bye.left', 'bye.right', 'please.left', 'please.right'])
# 'hello.left', 'hello.right', 'thank you.left', 'thank you.right', 'i love you.left', 'i love you.right', 'bye.left', 'bye.right', 'please.left', 'please.right'

In [48]:
for sign in signs:
    for video in range(30):
        try:
            os.makedirs(os.path.join(data, sign, str(video)))
        except:
            pass

In [49]:
capture = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while capture.isOpened():
        for sign in signs:
            for video in range(30):
                for frame_number in range(30):
                    return_value, frame = capture.read()
                    frame, result = mediapipe_detection(frame, holistic)
                    draw_landmarks(frame, result)
                    if frame_number == 0:
                        cv2.putText(frame, 'STARTING COLLECTION', (120, 200),
                                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 4, cv2.LINE_AA)
                        cv2.putText(frame, 'Collecting frames for {} Video Number {}'.format(sign, video), (15, 12),
                                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                        cv2.imshow("Mime by Ridhi and Kartikay", frame)
                        cv2.waitKey(2000)
                    else:
                        cv2.putText(frame, 'Collecting frames for {} Video Number {}'.format(sign, video), (15, 12),
                                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                        cv2.imshow("Mime by Ridhi and Kartikay", frame)
                    landmarks = extract_landmark(result)
                    np_path = os.path.join(data, sign, str(video), str(frame_number))
                    np.save(np_path, landmarks)
                    if cv2.waitKey(1) & 0xFF == ord('x'):
                        break
            capture.release()
            cv2.destroyAllWindows()

In [8]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [10]:
label_map = {label: num for num, label in enumerate(signs)}
sequences, labels = [], []
for sign in signs:
    for video in range(30):
        window = []
        for frame_number in range(30):
            res = np.load(os.path.join(data, sign, str(video), "{}.npy".format(frame_number)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[sign])

In [11]:
x = np.array(sequences)
y = to_categorical(labels).astype(int)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

In [9]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [15]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30, 1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(signs.shape[0], activation='softmax'))
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model.fit(x_train, y_train, epochs=2000, callbacks=[tb_callback])

Epoch 1/2000
8/8 [==============================] - 6s 95ms/step - loss: 3.1760 - categorical_accuracy: 0.1083
Epoch 2/2000
8/8 [==============================] - 1s 94ms/step - loss: 2.1239 - categorical_accuracy: 0.2625
Epoch 3/2000
8/8 [==============================] - 1s 93ms/step - loss: 2.4593 - categorical_accuracy: 0.1208
Epoch 4/2000
8/8 [==============================] - 1s 94ms/step - loss: 2.3049 - categorical_accuracy: 0.1292
Epoch 5/2000
8/8 [==============================] - 1s 94ms/step - loss: 2.2857 - categorical_accuracy: 0.0958
Epoch 6/2000
8/8 [==============================] - 1s 96ms/step - loss: 2.1958 - categorical_accuracy: 0.1208
Epoch 7/2000
8/8 [==============================] - 1s 95ms/step - loss: 2.1508 - categorical_accuracy: 0.1125
Epoch 8/2000
8/8 [==============================] - 1s 97ms/step - loss: 2.0431 - categorical_accuracy: 0.1708
Epoch 9/2000
8/8 [==============================] - 1s 94ms/step - loss: 1.9919 - categorical_accuracy: 0.2208
E

In [ ]:
#tensorboard --logdir=. run it in cmd, inside train folder

In [16]:
model.save('mime3.h5')

In [10]:
model = tf.keras.models.load_model('mime3.h5')

In [11]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_6 (LSTM)               (None, 30, 64)            442112    
                                                                 
 lstm_7 (LSTM)               (None, 30, 128)           98816     
                                                                 
 lstm_8 (LSTM)               (None, 64)                49408     
                                                                 
 dense_6 (Dense)             (None, 64)                4160      
                                                                 
 dense_7 (Dense)             (None, 32)                2080      
                                                                 
 dense_8 (Dense)             (None, 10)                330       
                                                                 
Total params: 596,906
Trainable params: 596,906
Non-tr

In [20]:
output = model.predict(x_test)
print(signs[np.argmax(output[4])])
print(signs[np.argmax(y_test[4])])

hello.left
hello.left


In [11]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [22]:
yhat = model.predict(x_test)
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [23]:
multilabel_confusion_matrix(ytrue, yhat)

array([[[50,  1],
        [ 0,  9]],

       [[56,  0],
        [ 0,  4]],

       [[54,  0],
        [ 0,  6]],

       [[55,  0],
        [ 0,  5]],

       [[54,  1],
        [ 0,  5]],

       [[57,  1],
        [ 0,  2]],

       [[52,  0],
        [ 2,  6]],

       [[52,  1],
        [ 0,  7]],

       [[56,  0],
        [ 0,  4]],

       [[50,  0],
        [ 2,  8]]], dtype=int64)

In [24]:
accuracy_score(ytrue, yhat)

0.9333333333333333

In [12]:
colors = [(245,117,16), (245,117,16), (117,245,16), (117,245,16), (16,117,245), (16,117,245), (117,245,16), (117,245,16), (245,117,16), (245,117,16)]
def prob_viz(res, signs, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, signs[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)  
    return output_frame

In [17]:
sequence = []
sentence = []
predictions = []
threshold = 0.5

cap = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        ret, frame = cap.read()
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        draw_landmarks(image, results)
        keypoints = extract_landmark(results)
        sequence.append(keypoints)
        sequence = sequence[-30:]
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(signs[np.argmax(res)])
            predictions.append(np.argmax(res))
            if np.unique(predictions[-10:])[0]==np.argmax(res): 
                if res[np.argmax(res)] > threshold: 
                    if len(sentence) > 0: 
                        if signs[np.argmax(res)] != sentence[-1]:
                            sentence.append(signs[np.argmax(res)])
                    else:
                        sentence.append(signs[np.argmax(res)])
            if len(sentence) > 5: 
                sentence = sentence[-5:]
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image,' ', sentence[-1], (3,30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        cv2.imshow('Mime by Ridhi and Kartikay', image)
        if cv2.waitKey(10) & 0xFF == ord('x'):
            break
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>


IndexError: list index out of range

In [14]:
sequence = []
sentence = []
predictions = []
threshold = 0.5

cap = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        ret, frame = cap.read()
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        draw_landmarks(image, results)
        keypoints = extract_landmark(results)
        sequence.append(keypoints)
        sequence = sequence[-30:]
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(signs[np.argmax(res)])
            predictions.append(np.argmax(res))
            if np.unique(predictions[-10:])[0]==np.argmax(res): 
                if res[np.argmax(res)] > threshold: 
                    if len(sentence) > 0: 
                        if signs[np.argmax(res)] != sentence[-1]:
                            sentence.append(signs[np.argmax(res)])
                    else:
                        sentence.append(signs[np.argmax(res)])
            if len(sentence) > 5: 
                sentence = sentence[-5:]
            image = prob_viz(res, signs, image, colors)
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        cv2.imshow('Mime by Ridhi and Kartikay', image)
        if cv2.waitKey(10) & 0xFF == ord('x'):
            break
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

IndexError: list index out of range

: 